In [1]:
import json

In [3]:
candidates = []

path = '/home/nazar/ParlAI/data/WoZ/'
fnames = ['woz_train_en.json', 'woz_validate_en.json', 'woz_test_en.json']

for fname in fnames:

    with open(path+fname, "r") as f:
        data = json.load(f)

    for dialogue in data:
        for line in dialogue['dialogue']:
            for el in [':'.join(turn_labels) for turn_labels in line['turn_label']]:
                candidates.append(el) 

In [4]:
candidates = set(candidates)

In [5]:
with open('labels_full.dict', 'a') as f:
    for el in candidates:
        f.write(el+'\n')

# dialog babi task 5 checking labels

In [2]:
labels = []
with open('../../../data/dialog-bAbI/dialog-bAbI-tasks/dialog-babi-task5-full-dialogs-trn.txt') as f:
    for line in f.readlines():
        if '\t' in line:
            strings = line.strip().split('\t')
            labels.append(
                strings[-1]
            )

In [3]:
labels = set(labels)

In [4]:
len(labels)

1098

In [14]:
len(labels)

1098

In [5]:
with open('task5/candidates.txt', 'a') as f:
    for el in labels:
        f.write(el+'\n')

In [1]:
from parlai.scripts.display_data import DisplayData

In [4]:
DisplayData.main(
    task='fromfile:parlaiformat',
    fromfile_datapath='flow_data/testflow',
    fromfile_datatype_extension=True
)

15:48:49 | Opt:
15:48:49 |     allow_missing_init_opts: False
15:48:49 |     batchsize: 1
15:48:49 |     datapath: /home/nazar/ParlAI/data
15:48:49 |     datatype: train:ordered
15:48:49 |     dict_class: None
15:48:49 |     display_add_fields: 
15:48:49 |     download_path: None
15:48:49 |     dynamic_batching: None
15:48:49 |     fromfile_datapath: flow_data/testflow
15:48:49 |     fromfile_datatype_extension: True
15:48:49 |     hide_labels: False
15:48:49 |     ignore_agent_reply: True
15:48:49 |     image_cropsize: 224
15:48:49 |     image_mode: raw
15:48:49 |     image_size: 256
15:48:49 |     init_model: None
15:48:49 |     init_opt: None
15:48:49 |     is_debug: False
15:48:49 |     loglevel: info
15:48:49 |     max_display_len: 1000
15:48:49 |     model: None
15:48:49 |     model_file: None
15:48:49 |     multitask_weights: [1]
15:48:49 |     mutators: None
15:48:49 |     num_examples: 10
15:48:49 |     override: "{'task': 'fromfile:parlaiformat', 'fromfile_datapath': 'flow_da

# Creating test set with greetings

In [ ]:
# text:hello	labels:hi, how are you?	episode_done:True

# text:hi	labels:hello hello	episode_done:True

In [4]:
response = 'Good day, [Sir/Madam]! Can I speak with [NAME], please?'

with open('flow_data/responses/greetings.txt') as greetings:
    with open('flow_data/testflow_train.txt', 'a') as dataset:
        for line in greetings.readlines():
            dataset.write(
                f'text:{line.strip()}\tlabels:{response}\tepisode_done:True\n\n'
            )
